# Pre-Processing Video Input

This part is a pre-procession of video, we extract 30 frames from each video and resize them in "Frame_Data" folder, the datasets of our deep network are all from this folder.

In [1]:
import cv2
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# capture all frames from a video
def capvideo(video, buffer):
    '''
    video: the path of video
    buffer: folder to store all frames
    '''
    cap = cv2.VideoCapture(video)
#     print(cap.isOpened())
    count = 1
    success = True
    while(success):
        success, frame = cap.read()
        cv2.imwrite(os.path.join(buffer, str(count)+".jpg"), frame)
        count += 1
    cap.release()
    return count

In [3]:
# choose 30 frames and crop them to 128*128
def crop(start, count, buffer, dst):
    '''
    start: start image
    count: result from capvideo()
    buffer: same buffer in capvideo()
    dst: path of the frames should be stored
    '''
    index = 1
    for i in np.linspace(start, count-1-start, 30, dtype=int):
        img = Image.open(os.path.join(buffer, str(i)+".jpg"))
        img = img.crop((320,40,960,680))
        img = img.resize((128,128), resample=Image.NEAREST)            
        img.save(os.path.join(dst, str(index)+".jpg"))
        index += 1

In [4]:
def rmbuffer(buffer):
    bufferlist = os.listdir(buffer)
    for item in bufferlist:
        os.remove(os.path.join(buffer, item))
    os.rmdir(buffer)

In [6]:
# Process on training data
video_path = "./Video_Data"
frame_path = "./Frame_Data"
dirlist = os.listdir(video_path)
for i in dirlist:
    current_path = os.path.join(video_path, i)
    current_dst = os.path.join(frame_path, i)
    if os.path.exists(current_dst == False):
        os.mkdir(current_dst)
    videolist = os.listdir(current_path)
    person = 1
    for item in videolist:
        if item.endswith(".MP4"):
            buffer = "./buffer"
            os.mkdir(buffer)
            count = capvideo(os.path.join(current_path, item), buffer)
            for start in range(1,4):
                dst = os.path.join(current_dst, "person_"+str(person))
                if os.path.exists(dst) == False:
                    os.mkdir(dst)
                crop(start, count, buffer, dst)
                person += 1
            rmbuffer(buffer)

In [8]:
# Process on query data
Qvideo_path = "./QVideo_Data"
Qframe_path = "./QFrame_Data"
if os.path.exists(Qframe_path) == False:
    os.mkdir(Qframe_path)

index = 1
for i in os.listdir(Qvideo_path):
    if i.endswith(".MP4"):
        buffer = "./buffer"
        os.mkdir(buffer)
        count = capvideo(os.path.join(Qvideo_path, i), buffer)
        for start in range(1,2):
            dst = os.path.join(Qframe_path, "query_"+str(index))
            if os.path.exists(dst) == False:
                os.mkdir(dst)
            crop(start, count, buffer, dst)
            index += 1
        rmbuffer(buffer)